In [2]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("yelp_data.db")
cursor = conn.cursor()

# Fetch IMDB reviews
cursor.execute("SELECT preprocessed_text FROM imdb_reviews LIMIT 5")
imdb_reviews = cursor.fetchall()

# Fetch Amazon reviews
cursor.execute("SELECT preprocessed_text FROM amazon_reviews LIMIT 5")
amazon_reviews = cursor.fetchall()

# Close connection
conn.close()

# Print sample
print("IMDB Sample:", imdb_reviews[0])
print("Amazon Sample:", amazon_reviews[0])


IMDB Sample: ('rented curiousyellow video store controversy surrounded first released also heard first seized custom ever tried enter country therefore fan film considered controversial really see myselfbr plot centered around young swedish drama student named lena want learn everything life particular want focus attention making sort documentary average swede thought certain political issue vietnam war race issue united state asking politician ordinary denizen stockholm opinion politics sex drama teacher classmate married menbr kill curiousyellow year ago considered pornographic really sex nudity scene far even shot like cheaply made porno countryman mind find shocking reality sex nudity major staple swedish cinema even ingmar bergman arguably answer good old boy john ford sex scene filmsbr commend filmmaker fact sex shown film shown artistic purpose rather shock people make money shown pornographic theater america curiousyellow good film anyone wanting study meat potato pun intended 

In [3]:
import json

# Load GoEmotions dataset
with open("goemotions_cleaned.json", "r") as file:
    goemotions_data = json.load(file)

# Load Yelp reviews dataset
with open("yelp_sample_preprocessed.json", "r") as file:
    yelp_data = json.load(file)

# Print sample
print("GoEmotions Sample:", goemotions_data[0])
print("Yelp Sample:", yelp_data[0])


GoEmotions Sample: {'text': "My favourite food is anything I didn't have to cook myself.", 'emotions': ['neutral'], 'preprocessed_text': 'favourite food cook'}
Yelp Sample: {'review_id': 'SlOJFEMxtID4KR68TOKrAg', 'business_id': 'x_OCBGTbSzwslCV8XyvkyA', 'stars': 5.0, 'date': '2018-08-24 05:11:47', 'text': 'Yum! What a great local restaurant that has made a name for itself because it serves delicious food. The snickerdoodle pancake is delicious! They specialize in various kinds of eggs Benedict and all are delicious.', 'preprocessed_text': 'yum great local restaurant made name serf delicious food snickerdoodle pancake delicious specialize various kind egg benedict delicious'}


In [15]:
import cohere
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import sqlite3
import json
from sklearn.metrics.pairwise import cosine_similarity
import json


c:\Users\smrit\miniconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
c:\Users\smrit\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
co = cohere.Client("3OX8jQ6maGWiw4KCbiQn95SS0t8aaDrhzwy318Xk")


tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
sarcasm_model = AutoModelForSequenceClassification.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")

In [30]:
# Load GoEmotions JSON
with open("goemotions_cleaned.json", "r") as f:
    goemotions_data = json.load(f)

# Load Yelp JSON
with open("yelp_sample_preprocessed.json", "r") as f:
    yelp_data = json.load(f)

# Fetch IMDB & Amazon Data from SQLite
conn = sqlite3.connect("yelp_data.db")
cursor = conn.cursor()

cursor.execute("SELECT preprocessed_text FROM imdb_reviews")
imdb_data = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT preprocessed_text FROM amazon_reviews")
amazon_data = [row[0] for row in cursor.fetchall()]

conn.close()

In [11]:
# Extract unique emotion labels from GoEmotions dataset
emotion_labels = set(emotion for entry in goemotions_data for emotion in entry["emotions"])
print(list(emotion_labels))  # Convert to list if needed


['curiosity', 'sadness', 'gratitude', 'grief', 'excitement', 'relief', 'fear', 'embarrassment', 'optimism', 'anger', 'nervousness', 'disappointment', 'neutral', 'realization', 'desire', 'amusement', 'approval', 'remorse', 'love', 'surprise', 'pride', 'joy', 'caring', 'annoyance', 'admiration', 'confusion', 'disapproval', 'disgust']


In [ ]:
# # Convert set to list before embedding
# emotion_labels = list(emotion_labels)  
# emotion_embeddings = co.embed(texts=emotion_labels, model="embed-english-light-v3.0",input_type="classification").embeddings
# print(emotion_embeddings)


[[0.028152466, -0.0317688, 0.009300232, -0.008552551, 0.008834839, 0.016723633, 0.051757812, 0.021514893, -0.03427124, 0.04156494, 0.036224365, -0.061553955, -0.09277344, 0.004032135, -0.0005669594, -0.0058174133, 0.07684326, -0.054656982, -0.12756348, -0.00023078918, 0.056854248, -0.064086914, -0.03704834, -0.05722046, -0.05206299, -0.031341553, -0.07305908, -0.04348755, 0.051879883, -0.058563232, -0.04776001, 0.031280518, -0.045684814, 0.0023956299, 0.034210205, 0.012550354, -0.02659607, 0.04547119, 0.011810303, 0.03741455, 0.0026798248, -0.03314209, -0.004470825, -0.025848389, 0.038604736, 0.045074463, -0.0128479, 0.053741455, 0.13916016, 0.055511475, -0.03555298, 0.031158447, -0.020751953, 0.029968262, -0.01108551, 0.075683594, -0.000849247, -0.029953003, -0.011138916, 0.0072288513, -0.0138168335, -0.038848877, -0.09967041, 0.019424438, -0.03262329, 0.12231445, 0.030517578, -0.035705566, -0.0692749, -0.00969696, 0.09051514, 0.029434204, 0.047576904, 0.08459473, -0.0024089813, -0.02

In [45]:
def detect_sarcasm(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sarcasm_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs[0][1].item()  # Sarcasm score

In [37]:
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_name)

In [38]:
import numpy as np

def classify_emotions(text):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = emotion_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]

    emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
    
    top_emotions = sorted(zip(emotion_labels, probs.tolist()), key=lambda x: x[1], reverse=True)[:3]
    
    emotion_results = {}
    for emotion, score in top_emotions:
        if score > 0.75:
            activation = "High"
        elif score > 0.5:
            activation = "Medium"
        else:
            activation = "Low"
        emotion_results[emotion] = {"activation": activation, "intensity": round(score, 2)}

    return emotion_results



In [40]:
def process_text(text):
    sarcasm_score = detect_sarcasm(text)
    emotions = classify_emotions(text)

    # Reduce joy score for neutral/mixed reviews
    if "joy" in emotions and "neutral" in emotions:
        if emotions["neutral"]["intensity"] > 0.3:
            emotions["joy"]["intensity"] -= 0.2  # Reduce joy slightly
            emotions["neutral"]["intensity"] += 0.2  # Increase neutral

    # Ensure negative reviews are not misclassified as joyful or surprising
    if sarcasm_score < 0.3:  # Non-sarcastic but possibly negative review
        if "joy" in emotions and "sadness" in emotions:
            if emotions["sadness"]["intensity"] > 0.4:
                emotions["joy"]["intensity"] -= 0.3  # Reduce joy in negative reviews
                emotions["disgust"] = {"activation": "Medium", "intensity": 0.5}  # Add disgust

    return {
        "original_text": text,
        "sarcasm_score": round(sarcasm_score, 2),
        "emotions": emotions
    }


In [ ]:
all_reviews = [str(review) for review in (yelp_data + imdb_data + amazon_data) if isinstance(review, str)]
results = [process_text(review) for review in all_reviews[:100]]


for idx, res in enumerate(results[:5]):
    print(f"\nReview {idx+1}:")
    print("Original Text:", res["original_text"])
    #print("Sarcasm Score:", res["sarcasm_score"])
    print("Detected Emotions:", res["emotions"])



Review 1:
Original Text: rented curiousyellow video store controversy surrounded first released also heard first seized custom ever tried enter country therefore fan film considered controversial really see myselfbr plot centered around young swedish drama student named lena want learn everything life particular want focus attention making sort documentary average swede thought certain political issue vietnam war race issue united state asking politician ordinary denizen stockholm opinion politics sex drama teacher classmate married menbr kill curiousyellow year ago considered pornographic really sex nudity scene far even shot like cheaply made porno countryman mind find shocking reality sex nudity major staple swedish cinema even ingmar bergman arguably answer good old boy john ford sex scene filmsbr commend filmmaker fact sex shown film shown artistic purpose rather shock people make money shown pornographic theater america curiousyellow good film anyone wanting study meat potato pu

In [41]:
# Test examples with sarcasm and non-sarcasm cases
test_reviews = [
    "Oh wow, this service is just fantastic... not!",  # Sarcastic
    "I love how my package arrived 5 days late, great job!",  # Sarcastic
    "This movie was absolutely stunning! I enjoyed every moment.",  # Non-sarcastic
    "I'm so happy with my purchase, the quality is amazing.",  # Non-sarcastic
    "Sure, because waiting in line for an hour is exactly how I wanted to spend my day.",  # Sarcastic
    "The restaurant had the best food I've ever tasted. Highly recommend!",  # Non-sarcastic
    "Oh great, another update that makes the app even worse.",  # Sarcastic
    "The hotel staff was extremely kind and helpful. Would stay again!",  # Non-sarcastic
]

# Run the test cases through the pipeline
test_results = [process_text(review) for review in test_reviews]

# Print results
for idx, res in enumerate(test_results):
    print(f"\nTest Case {idx+1}:")
    print("Original Text:", res["original_text"])
    print("Sarcasm Score:", res["sarcasm_score"])
    print("Detected Emotions:", res["emotions"])



Test Case 1:
Original Text: Oh wow, this service is just fantastic... not!
Sarcasm Score: 0.82
Detected Emotions: {'surprise': {'activation': 'High', 'intensity': 0.96}, 'joy': {'activation': 'Low', 'intensity': 0.02}, 'sadness': {'activation': 'Low', 'intensity': 0.01}}

Test Case 2:
Original Text: I love how my package arrived 5 days late, great job!
Sarcasm Score: 0.55
Detected Emotions: {'joy': {'activation': 'Medium', 'intensity': 0.74}, 'surprise': {'activation': 'Low', 'intensity': 0.18}, 'sadness': {'activation': 'Low', 'intensity': 0.05}}

Test Case 3:
Original Text: This movie was absolutely stunning! I enjoyed every moment.
Sarcasm Score: 0.55
Detected Emotions: {'joy': {'activation': 'High', 'intensity': 0.87}, 'surprise': {'activation': 'Low', 'intensity': 0.08}, 'neutral': {'activation': 'Low', 'intensity': 0.03}}

Test Case 4:
Original Text: I'm so happy with my purchase, the quality is amazing.
Sarcasm Score: 0.34
Detected Emotions: {'joy': {'activation': 'High', 'inte

In [47]:
# Define test reviews (non-sarcastic)
non_sarcastic_reviews = [
    "The customer service at this restaurant was exceptional! The staff was polite, attentive, and made sure we had everything we needed. I’ll definitely be coming back!",
    "The product arrived on time and in good condition. It works as expected, though there’s nothing particularly special about it. A decent purchase overall.",
    "Unfortunately, this phone did not meet my expectations. The battery drains quickly, and the camera quality is below average. I wouldn’t recommend it for the price.",
    "The hotel room was spacious and clean, but the air conditioning was broken, making it uncomfortable to sleep at night. The staff was helpful, but they couldn't fix the issue in time.",
    "This book was an absolute masterpiece! The storytelling was engaging, and I couldn’t put it down. One of the best books I’ve read this year!"
]

# Run the test cases through the pipeline
results = [process_text(review) for review in non_sarcastic_reviews]

# Print results
for idx, res in enumerate(results):
    print(f"\nTest Case {idx+1}:")
    print("Original Text:", res["original_text"])
    print("Sarcasm Score:", res["sarcasm_score"])
    print("Detected Emotions:", res["emotions"])



Test Case 1:
Original Text: The customer service at this restaurant was exceptional! The staff was polite, attentive, and made sure we had everything we needed. I’ll definitely be coming back!
Sarcasm Score: 0.17
Detected Emotions: {'joy': {'activation': 'High', 'intensity': 0.87}, 'neutral': {'activation': 'Low', 'intensity': 0.09}, 'surprise': {'activation': 'Low', 'intensity': 0.01}}

Test Case 2:
Original Text: The product arrived on time and in good condition. It works as expected, though there’s nothing particularly special about it. A decent purchase overall.
Sarcasm Score: 0.38
Detected Emotions: {'joy': {'activation': 'Medium', 'intensity': 0.39999999999999997}, 'neutral': {'activation': 'Low', 'intensity': 0.55}, 'surprise': {'activation': 'Low', 'intensity': 0.03}}

Test Case 3:
Original Text: Unfortunately, this phone did not meet my expectations. The battery drains quickly, and the camera quality is below average. I wouldn’t recommend it for the price.
Sarcasm Score: 0.32